In [24]:
from __future__ import unicode_literals
import pandas as pd 
import numpy as np
import os 

from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

from string import punctuation
from collections import Counter , defaultdict

from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

from tabulate import tabulate
import gensim
import struct
from gensim.models.word2vec import Word2Vec

In [25]:
data = pd.read_csv('review.csv')

In [3]:
DataX = data['review']
DataY = data['sentiment']
y = DataY

In [4]:
data.head()

,review,sentiment
0,"lancelot du lac ( lancelot du lac ) ( france ,...",1
1,director : brian de palma writer : david koepp...,1
2,"six days , seven nights reviewed by jamie peck...",1
3,"cast : mel gibson ( jerry fletcher ) , julia r...",1
4,"all great things come to an end , and the dot-...",1


In [5]:
def String_Splitter(data):
    """
    
    """
    for i in range(len(data)):
        try :             
            data[i]=data[i].split()
        except:
            print(i)
            data[i] = data[i]
    return data




def wordToString(wordList):
    """
    
    """
    stringList = []
    for i in range(0,len(wordList)):
        stringList.append(' '.join(word for word in wordList[i]))
    return stringList




def TextCleaner(Data, stopwordsList= ''):
    """
    
    """
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    dataList = Data
    table = str.maketrans('', '', punctuation)
    CountVector = []
    
    for i in range(0, len(dataList)):
        vocabulary = []
        
        for j in range(0, len(dataList[i])):
            dataList[i][j] = stemmer.stem(dataList[i][j])
            dataList[i][j] = lemmatizer.lemmatize(dataList[i][j])
            
        dataList[i] = [word for word in dataList[i] if word.isalpha()]
        dataList[i] = [w.translate(table) for w in dataList[i]]
        dataList[i] = [word for word in dataList[i] if len(word) > 3]
        vocabulary.append(dataList[i])
        
    return dataList

In [6]:
temp_corpus = DataX.values.tolist()

i = len(temp_corpus)-1
while i != 0:
    if temp_corpus[i]==float('nan'):
        del(temp_corpus[i])
    i-=1
    

new_data = String_Splitter(temp_corpus)
X = TextCleaner(new_data)

In [7]:
X[:1]

[['lancelot',
  'lancelot',
  'franc',
  'film',
  'review',
  'jame',
  'kendrick',
  'director',
  'robert',
  'bresson',
  'screenwrit',
  'robert',
  'bresson',
  'base',
  'epic',
  'poem',
  'chretien',
  'troy',
  'star',
  'simon',
  'lancelot',
  'laura',
  'duke',
  'condomina',
  'queen',
  'guinever',
  'vladimir',
  'king',
  'arthur',
  'humbert',
  'balsan',
  'gawain',
  'patrick',
  'bernhard',
  'modr',
  'arthur',
  'montalembert',
  'lionel',
  'mpaa',
  'rate',
  'rate',
  'review',
  'ench',
  'director',
  'robert',
  'bresson',
  'lancelot',
  'lake',
  'achiev',
  'exact',
  'opposit',
  'effect',
  'thoma',
  'malori',
  'mort',
  'darthur',
  'instead',
  'enshrin',
  'legend',
  'king',
  'arthur',
  'knight',
  'round',
  'tabl',
  'dethron',
  'them',
  'reveal',
  'arthur',
  'weak',
  'ineffectu',
  'leader',
  'knight',
  'group',
  'jealous',
  'bicker',
  'fail',
  'live',
  'legend',
  'prescrib',
  'them',
  'chivalri',
  'place',
  'lancelot',
  'l

In [8]:
whole_corpora = ""
for i in range(len(data)): 
    whole_corpora = whole_corpora + " " + data.iloc[i]['review'] 

In [9]:
word_embedding_path = "cc.en.300.vec"
encoding = "utf-8"
with open(word_embedding_path, "rb") as lines:
    wvec = {line.split()[0].decode(encoding): np.array(line.split()[1:],dtype=np.float32)
               for line in lines}

In [15]:
glove_small = {}
all_words = set(w for words in X for w in words) # Making a dictionary of mutual words
                                         # in the corpora and the word embedding model.
with open(word_embedding_path, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        if (word in all_words):
            nums=np.array(parts[1:], dtype=np.float32)
            glove_small[word] = nums

In [16]:
# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
model = Word2Vec(X, size=100, window=5, min_count=5, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

c:\users\mahdi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [11]:
# start with the classics - naive bayes of the multinomial and bernoulli varieties
# with either pure counts or tfidf features
mult_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
mult_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
# SVM - which is supposed to be more or less state of the art 
# http://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf
svc = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])
svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])

In [12]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    
# and a tf-idf version of the same
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [17]:
MEV = MeanEmbeddingVectorizer(glove_small)
MEV.fit(X,y)
feature_matrix = MEV.transform(X)



TFEV = TfidfEmbeddingVectorizer(glove_small)
TFEV.fit(X,y)
TF_feature_matrix = TFEV.transform(X)

In [18]:
feature_matrix[0]

array([-1.31113054e-02, -3.05542577e-04, -1.23092420e-02,  6.84455112e-02,
       -1.64194051e-02,  2.39376514e-03,  1.01905307e-02, -4.71686013e-03,
       -7.15057459e-03, -4.93094837e-03, -2.76561007e-02, -2.07378808e-02,
       -3.55912326e-03, -1.45734465e-02, -3.65196448e-03,  1.62067004e-02,
        1.33644333e-02, -6.67690812e-03, -1.08526573e-02,  4.32840968e-03,
        5.42240264e-03,  1.17348656e-02,  1.30667435e-02, -1.18037069e-03,
       -1.82498787e-02, -2.13859156e-02, -1.15207853e-02, -6.32331765e-04,
       -8.90530925e-03,  9.10438523e-02,  3.65196168e-03,  3.03879846e-03,
       -1.02831442e-02, -2.09106337e-02, -3.04665091e-03,  4.89145471e-03,
        2.43348535e-03,  2.51489710e-02, -5.20969648e-03,  1.51732238e-03,
        5.13164047e-03, -7.08013913e-03, -1.27316425e-02, -1.08983507e-03,
       -2.98064668e-02,  2.37358026e-02, -1.18117779e-02,  8.20023380e-03,
       -1.66136343e-02,  4.21662582e-03,  1.26738949e-02, -4.63349186e-03,
        1.93697549e-02, -

In [19]:
TF_feature_matrix[0]

array([-4.03867811e-02, -7.37054227e-03, -6.70127720e-02,  2.41524920e-01,
       -9.57998913e-03,  1.39007065e-03,  3.56517285e-02, -2.35017464e-02,
       -3.80988009e-02, -3.52753066e-02, -1.04767896e-01, -8.23728815e-02,
       -2.10841354e-02, -6.51720986e-02, -2.59066727e-02,  7.23229647e-02,
        5.51743098e-02, -1.11048175e-02, -3.30254994e-02,  6.69181021e-03,
        4.40071523e-02,  3.52497138e-02,  3.52309458e-02, -4.80314065e-03,
       -2.40648109e-02, -4.69535477e-02, -4.63568792e-02,  7.39399157e-03,
       -2.81552095e-02,  3.96615952e-01, -1.66295224e-03,  1.65128242e-02,
       -4.50002924e-02, -2.60777529e-02, -1.87330022e-02, -3.59837222e-03,
        2.35757176e-02,  9.53708068e-02, -1.16779655e-02,  1.26356725e-02,
        2.58193891e-02, -3.33762392e-02, -4.46071103e-02, -1.14148641e-02,
       -1.27179563e-01,  7.67266154e-02, -3.43475789e-02,  3.31966467e-02,
       -7.66736567e-02, -1.38697736e-02,  2.05815844e-02, -5.03384694e-02,
        7.75212049e-02, -

In [20]:
# Extra Trees classifier is almost universally great, let's stack it with our embeddings
etree_glove_small = Pipeline([("glove vectorizer", MeanEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_glove_small_tfidf = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])


etree_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])


In [26]:
all_models = [
    ("mult_nb", mult_nb),
    ("mult_nb_tfidf", mult_nb_tfidf),
    ("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    ("svc", svc),
    ("svc_tfidf", svc_tfidf),
    ("w2v", etree_w2v),
    ("w2v_tfidf", etree_w2v_tfidf),
    ("glove_small", etree_glove_small),
    ("glove_small_tfidf", etree_glove_small_tfidf),

]


unsorted_scores = [(name, cross_val_score(model, X, y, cv=5).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])


print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

model                score
-----------------  -------
svc_tfidf           0.8297
mult_nb_tfidf       0.8023
mult_nb             0.7980
svc                 0.7857
bern_nb             0.7641
bern_nb_tfidf       0.7641
glove_small         0.7071
glove_small_tfidf   0.6876
w2v                 0.5808
w2v_tfidf           0.5758
